In this file we will use the Envirocar data to estimate the traffic density, leading to a kind of heatmap. A naive way to approach this problem is to take our EnviroCar points and check for each of them in which street segment of OSM they lie. There should be no unmatched points except those on high-speed streets or leading outside the bounding box. After that we can sum up the unique track IDs per segment, which will represent our density function. This approach is (probably) naive because of runtime, so any approach that reduces point load should be undertaken. That could be



*   filter points based on speed
*   filter points based on bounding box
*   discretize EnviroCar data (measuring interval)




# LOAD DATA
For better accessibility all data is saved as shape files (run EnviroCar_get_data if data is missing). For our traffic density analysis we need to match the envirocar data to some background map to perform aggregation on. This will be data downloaded from OSM. A metadata description can be found [here](http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf). We only need the shapefile containing roads and paths.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import pandas as pd
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Polygon, Point
import os
import rtree

In [ ]:
filepath = os.path.join(os.getcwd(), "data")

In [ ]:
# while loading the road data (OSM) we can define a bbox -> other data won't be loaded
bbox_osm = (7.55, # min_x
            51.875, # min_y
            7.7200, # max_x 
            52.025)  # max_y

roads = gpd.read_file(os.path.join(filepath, "OSM_muenster", "roads.shp"), bbox=bbox_osm)

# should work outside of Colab (but needs to be checked)
#roads = gpd.read_file("data/OSM_muenster/roads.shp", bbox=bbox_osm)

# check dimensions for better data feel
roads.shape

In [ ]:
# to group road segments later, we need a unique id, for now this will just a consecutive number
roads["ID"] = [x for x in range(roads.shape[0])]

In [ ]:
# track data can be read using the same bbox

# large dataset (may take some time)
tracks = gpd.read_file(os.path.join(filepath, "envirocar_muenster", "envirocar_muenster_2000.shp"), bbox=bbox_osm)

# check dimensions for better data feel
tracks.shape

In [ ]:
tracks.set_crs("EPSG:4326", inplace = True)

In [ ]:
# plot both to verify correct loading
# first prepare 1x2 canvas
fig, ax = plt.subplots(1, 2, figsize=(15, 20))

roads.plot(ax=ax[0])
cx.add_basemap(ax=ax[0], crs=roads.crs.to_string())
ax[0].set_title("OSM data")

tracks.plot(ax=ax[1])
cx.add_basemap(ax=ax[1], crs=tracks.crs.to_string())
ax[1].set_title("EnviroCar data")

# FILTER DATA
We have already filtered both datasets according to our bounding box. We need to filter the OSM data to include only "Innenstadtverkehr". This can be achieved using the "fclass" feature. We will only include "primary", "secondary", "tertiary", "unclassified", "residential" and "living_street".

We will filter the EnviroCar data to include only measurements with less than 65 km/h. This way we want also want to eliminate any data that is not "Innenstadtverkehr". Also we regard only tracks within the city borders of Münster.

In [ ]:
# filter OSM road data
roads_filtered = roads[roads["type"].isin(["primary", "secondary", "tertiary", "unclassified", "residential", "living_street"])]
roads_filtered.shape

In [ ]:
# filter EnviroCar data

#Take only tracks inside the city borders of Münster
muenster=gpd.read_file(os.path.join(filepath, "stadtgebiet_muenster", "stadtgebiet.shp"))

muenster=muenster.to_crs(4326) # Adjust the coordinate reference system

tracks_filtered=tracks.within(muenster["geometry"][0]) # Geopandas within function checks if points are in polygon

#Take only tracks with speed value lower than 65 km/h
tracks_filtered = tracks[tracks["Speed.valu"] < 65]
tracks_filtered.shape

We have lost quite a lot of road segments, not that many Envirocar data points. Another odd thing though are a couple of straight lines in the track data, so we will filter those as well using a Polygon.

In [ ]:
# polygon coordinates will be in EPSG 3857
roads_filtered = roads_filtered.to_crs(epsg=3857)

In [ ]:
# There are some weird roads in the data, we can exclude them by checking for intersection with suitable polygons
polygon=Polygon(((825000,6.78e6), (825000,6.785e6), (830000,6.785e6),(830000,6.78e6)))
polygon2=Polygon(((848950,6.7908e6),(848950,6.7910e6),(849000,6.7910e6), (849000,6.7908e6))) # Some more weird roads that remained
roads_filtered = roads_filtered.loc[roads_filtered["geometry"].intersects(polygon)==False]
roads_filtered = roads_filtered.loc[roads_filtered["geometry"].intersects(polygon2)==False]

In [ ]:
# fix data type of temporal data for better handling
tracks_filtered["time"] = pd.to_datetime(tracks_filtered["time"])

In [ ]:
# filter data to only include tracks from 2018 (because verification data from Münster is also from 2018)
tracks_filtered = tracks_filtered[tracks_filtered["time"].dt.year == 2018]

# AMENDING DATA
For aggregating the data later on in the way we desire, we need to introduce a temporal feature (i.e. track on weekday or on weekend?).

In [ ]:
# now introduce new weekday feature
tracks_filtered["weekday"] = tracks_filtered["time"].dt.dayofweek

# 0-4 are weekdays, 5 & 6 are saturday and sunday
tracks_filtered["weekday"] = tracks_filtered["weekday"] < 5
tracks_filtered.head()

# BUFFERING
We will now for each street in roads_filtered collect and count the points of our EnviroCar data with unique track ID. For that we will first need a buffer around streets because of GPS and measuring inaccuracy. Our current map is in degrees (WGS 84 / EPSG:4326). If we want to add a buffer of 10 meters, we have to convert it to a projection in meters because the geopanda library cannot handle different units. One such a projection is WGS 84 / EPSG: 3857 which is also already used by contextily background maps (see [here](https://epsg.io/3857)).

We will also use cap_style = 2 for our buffer (see [here](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html)) to avoid counting traffic multiple times on intersections.

In [ ]:
# convert our data
tracks_filtered = tracks_filtered.to_crs(epsg=3857)
roads_filtered = roads_filtered.to_crs(epsg=3857)
muenster= muenster.to_crs(3857)

In [ ]:
# now add the buffer of 10 meters
# we will create a new object because one df cannot have two geometry columns and further processing may be bound to that name
roads_buffered = roads_filtered.copy(deep=True)
roads_buffered["geometry"] = roads_buffered["geometry"].buffer(distance=10, cap_style=2)
roads_buffered.head()

In [ ]:
# to plot change everything back to 4326
tracks_filtered = tracks_filtered.to_crs(epsg=4326)
roads_filtered = roads_filtered.to_crs(epsg=4326)
muenster= muenster.to_crs(4326)
roads_buffered = roads_buffered.to_crs(epsg=4326)

In [ ]:
# print to see difference (using zoom)
fig, ax = plt.subplots(1, 2, figsize=(10, 20))

# roads without buffers
roads_filtered["geometry"].plot(ax=ax[0])
ax[0].set_title("OSM data without buffer")
ax[0].set_xlim(7.6062, 7.6148)
ax[0].set_ylim(51.9636, 51.9688)

roads_buffered["geometry"].plot(ax=ax[1])
ax[1].set_title("OSM data with buffer")
ax[1].set_xlim(7.6062, 7.6148)
ax[1].set_ylim(51.9636, 51.9688)

Buffering has worked as expected.

In [ ]:
# change back to 3857 to work with the rest of the script
tracks_filtered = tracks_filtered.to_crs(epsg=3857)
roads_filtered = roads_filtered.to_crs(epsg=3857)
muenster= muenster.to_crs(3857)
roads_buffered = roads_buffered.to_crs(epsg=3857)

# AGGREGATE
Now that we have our buffers, we can look for intersections between our points and the buffered streets. As points are small, intersects or within/contains should not make a difference, but just to be sure we will use intersect. An efficient way to loop over the dataframe will be to perform a spatial join, which can easily be done using geopandas.

In [ ]:
# to keep all the roads data, we will perform right outer join -> all points should be matched (at least) once though
intersections = gpd.sjoin(tracks_filtered, roads_buffered, how='right')
intersections.head()

We will now group by streets segment and then coun the unique track IDs for each segment. We can make our traffic density estimation more precise by separating weekdays and weekends.

In [ ]:
# the unique key of each segment is the ID we have given in the beginning, so we can use this for grouping (and separate by weekday / weekend)
# then we look for unique track IDs and count them per group (rename for convenience)
num_tracks = intersections.groupby(["ID", "weekday"])["track.id"].nunique().rename("num_tracks")
num_tracks.head()

In [ ]:
# now we need to pivot this df, so that we have the ID as index and then two columns with the number of tracks, one "num_tracks_weekday" and one "num_tracks_weekend"

# first convert series to DF
num_tracks = num_tracks.to_frame()

# now unstack
num_tracks = num_tracks.unstack()
num_tracks

In [ ]:
# now rename columns and fill missing values with 0
num_tracks.columns = ["num_tracks_weekend", "num_tracks_weekday"]
num_tracks = num_tracks.fillna(0)
num_tracks.head()

In [ ]:
# we can now join back our counts to the OSM road data (using id)
results = roads_filtered.join(num_tracks, on="ID", how="left")

In [ ]:
results.head()

In [ ]:
results.describe()

In [ ]:
# we can see from stats above, that not all streets have been matched to tracks -> we need to fill those with 0s
results[["num_tracks_weekend", "num_tracks_weekday"]] = results[["num_tracks_weekend", "num_tracks_weekday"]].fillna(0)
results.head()

In [ ]:
results.describe()

In [ ]:
# finally we can calculate the total track count from weekend and weekday counts
results["num_tracks_total"] = results["num_tracks_weekend"] + results["num_tracks_weekday"]
results.head()

In [ ]:
# save results
results.to_file(os.path.join(filepath, "results", "muenster_2018_est_traffic_density.shp"))

# PLOTTING RESULTS

In [ ]:
# to see more clearly, exclude streets with no tracks on them
results=results.loc[results["num_tracks_total"] > 0]

## Total amount of tracks
We have now the number of tracks per road segment. We will try to color-code this metric and plot it on a map for analysis. We will start using only the total amount of tracks.

In [ ]:
# we can specify column to be used for coloring -> in our case num_tracks
# we can also apply a colormap to color roads depending on value (important to choose sequential one because num_tracks is sequential)
# background map will be black and white for easier visibility of roads

# prepare canvas for plots and colorbar
fig, ax = plt.subplots(1, 2, figsize=(15, 20))
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.1)

# first plot only track data points
tracks_filtered.plot(ax=ax[0], color="#08519c")
#cx.add_basemap(ax=ax[0], crs=tracks_filtered.crs.to_string(), source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[0].set_title("EnviroCar track data")

# Plot city borders
muenster['geometry'].plot(ax=ax[0], facecolor="none",edgecolor="black")

# now plot number of tracks per segment
results.plot(ax=ax[1], column="num_tracks_total", cmap="Blues", legend=True, cax=cax)
#cx.add_basemap(ax=ax[1], crs=results.crs.to_string(), source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[1].set_title("Traffic density estimation using \n aggregated EnviroCar track data")

plt.tight_layout()

We can see that the maximum amount of tracks (~70) dominates the colorbar. These may be outliers though.

In [ ]:
# check dsitribution of num_tracks
results["num_tracks_total"].describe()

In [ ]:
# plot distribution
results["num_tracks_total"].plot.box(vert=False, figsize=(8, 3))

We can see that we have a couple of (extreme) outliers, as most of num_track entries are below 10. We can give the plotting function a maximum value for our colormap, let's take 20 (because that's about where the top whisker ends). Another thing that we can try is to discretize our values into bins. Either a fixed amount of bins with the same number of entries (pd.cut; bin length will vary) or a fixed amount of bins of the same length (pd.qcut; number of bin entries will vary). Let's try both to see what works best. We will use 5 bins.

In [ ]:
# first bins with bins of the same width
bins_equal_width = pd.cut(results["num_tracks_total"], bins=5)

# then bin with bins containing the same amount of entries
bins_equal_entries = pd.qcut(results["num_tracks_total"], q=5, duplicates="drop")

In [ ]:
bins_equal_width

In [ ]:
bins_equal_entries

In [ ]:
# now plot -> first prepare 2x2 canvas
fig, ax = plt.subplots(2, 2, figsize=(15, 15))
ax = ax.flatten()

# first plot original results
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[0], column="num_tracks_total", cmap="Blues", legend=True, cax=cax)
#cx.add_basemap(ax=ax[0], crs=tracks_filtered.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[0].set_title("Traffic density estimation using \n original data")

# then plot results using vmax = 10
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[1], column="num_tracks_total", cmap="Blues", legend=True, cax=cax, vmax=10)
#cx.add_basemap(ax=ax[1], crs=tracks_filtered.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[1].set_title("Traffic density estimation using \n vmax = 20")

# now using bins with equal entries
results.plot(ax=ax[2], column=bins_equal_entries, cmap="Blues", legend=True, categorical=True)
#cx.add_basemap(ax=ax[2], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[2].set_title("Traffic density estimation using \n bins with equal amount of entries")

# now using bins with same width
results.plot(ax=ax[3], column=bins_equal_width, cmap="Blues", legend=True, categorical=True)
#cx.add_basemap(ax=ax[3], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[3].set_title("Traffic density estimation using \n bins with equal width")

Using real DTV values of Münster, we would be able to check which approach is most suitable.

## Applying nonlinear colormap

In [ ]:
t=results['num_tracks_total']

class nlcmap(object):
    def __init__(self, cmap, levels):
        self.cmap = cmap
        self.N = cmap.N
        self.monochrome = self.cmap.monochrome
        self.levels = np.asarray(levels, dtype='float64')
        self._x = self.levels
        self.levmax = self.levels.max()
        self.transformed_levels = np.linspace(0.0, self.levmax,
             len(self.levels))

    def __call__(self, xi, alpha=1.0, **kw):
        yi = np.interp(xi, self._x, self.transformed_levels)
        return self.cmap(yi / self.levmax, alpha)

tmax = max(t)
#the choice of the levels depends on the data:
levels = np.concatenate((
   # [0, tmax],
    np.array([1,2,3,4,5,6,7,8,9]),
    np.linspace(10, tmax, 4), # Hier anpassen
    ))

levels = levels[levels <= tmax]
levels.sort()

cmap_nonlin = nlcmap(plt.cm.plasma, levels) #OrRd

In [ ]:

fig, ax=plt.subplots(1,1,figsize=(10,12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

results.plot(ax=ax, column="num_tracks_total", cmap=cmap_nonlin.cmap, cax=cax)
#cx.add_basemap(ax=ax, crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax.set_title("Traffic density estimation using nonlinear colormap")

sm = plt.cm.ScalarMappable(cmap=plt.cm.plasma, norm=plt.Normalize(vmin=0, vmax=tmax))
sm._A = []

cbar = fig.colorbar(sm, cax=cax)
#here we relabel the linear colorbar ticks to match the nonlinear ticks
cbar.set_ticks(cmap_nonlin.transformed_levels)
cbar.set_ticklabels(["%.2f" % lev for lev in levels])

## Tracks separated by weekend / weekday
We will compare our estimated traffic densitiies for the weekend, weekdays and the total counts. For mapping we will use vmax to structure the colorbar, as this seems the most realistic assumption (everything above vmax = 10 is considered haevy traffic and does not need to be distinguished any further).

In [ ]:
# let's take a look at the distributions first
ax = results[["num_tracks_total", "num_tracks_weekday", "num_tracks_weekend"]].boxplot(figsize=(20, 5), vert=False)
ax.set_title("Distribution of estimated traffic density", fontsize=20)

We can see extreme outliers for all three kinds of track counts. However the maximum of the weekday and weekend data is less than the total (to be expected). The max of the weekend is even less than the weekday traffic.

In [ ]:
# same plot but now we filter all streets that have no tracks on them -> this way the other values won't be as skewed
# we can't filter data as it is, because we can only eliminate complete rows -> so first stack (and remove num_tracks from index)
only_tracks = results[["num_tracks_total", "num_tracks_weekday", "num_tracks_weekend"]].stack().reset_index(level=1)

# rename for convenience
only_tracks.columns = ["track_type", "value"]
only_tracks.head()

In [ ]:
# now we can easily filter and then do a grouped boxplot
ax = only_tracks[only_tracks["value"] > 0].boxplot(by="track_type", vert=False, figsize=(20, 5), labels=None)
ax.set_title("Distribution of estimated traffic density", fontsize=20)
plt.suptitle("")

The data is still cluttered around the lower edge of the value range. We can still see that the amount of tracks on a segment during the weekend varies less than during the week. The boxes for weekday and total look almost the same.

In [ ]:
# prepare canvas first
fig, ax = plt.subplots(1, 3, figsize=(30, 10))

# plot total count
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[0], column="num_tracks_total", cmap="Blues", legend=True, cax=cax, vmax=10)
#cx.add_basemap(ax=ax[0], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[0].set_title("Traffic density estimation using total track counts")

# plot weekday count
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[1], column="num_tracks_weekday", cmap="Blues", legend=True, cax=cax, vmax=10)
#cx.add_basemap(ax=ax[1], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[1].set_title("Traffic density estimation using weekday track counts")

# plot weekend count
divider = make_axes_locatable(ax[2])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[2], column="num_tracks_weekend", cmap="Blues", legend=True, cax=cax, vmax=10)
#cx.add_basemap(ax=ax[2], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[2].set_title("Traffic density estimation using weekend track counts")

fig.suptitle("Comparison of estimated traffic density using the same colormap (vmax=20)", fontsize=25)

We can see that the weekend traffic is less heave than during the week. The roads with the most traffic stay the same though, with a couple of exceptions. Another way to look at this is using individual vmax values (taken from the distributions, the upper whisker in the box plot = 75% + 1.5 * (75% - 25%)).

In [ ]:
results.describe()

In [ ]:
# prepare canvas first
fig, ax = plt.subplots(1, 3, figsize=(30, 10))

# plot total count (vmax = 11)
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[0], column="num_tracks_total", cmap="Blues", legend=True, cax=cax, vmax=11)
#cx.add_basemap(ax=ax[0], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[0].set_title("Traffic density estimation using total track counts")

# plot weekday count (vmax = 10)
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[1], column="num_tracks_weekday", cmap="Blues", legend=True, cax=cax, vmax=10)
#cx.add_basemap(ax=ax[1], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[1].set_title("Traffic density estimation using weekday track counts")

# plot weekend count (vmax = 5)
divider = make_axes_locatable(ax[2])
cax = divider.append_axes("right", size="5%", pad=0.1)
results.plot(ax=ax[2], column="num_tracks_weekend", cmap="Blues", legend=True, cax=cax, vmax=5)
#cx.add_basemap(ax=ax[2], crs=results.crs, source=cx.providers.OpenStreetMap.BlackAndWhite)
ax[2].set_title("Traffic density estimation using weekend track counts")

fig.suptitle("Comparison of estimated traffic density using individual colormaps", fontsize=25)

We can see the same trends as above, but this time more clearly. Some streets are well frequented on weekdays and weekends, but some also relatively more heavily only on weekends (click on picture to see better). However, the absolute traffic level is less.